In [6]:
# Import the model we are using
#from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from matplotlib.widgets import Lasso
import numpy as np
import pandas as pd
from sklearn.linear_model import BayesianRidge, LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import warnings
import pickle
from xgboost import XGBRegressor
warnings.filterwarnings('ignore')

# Pandas is used for data manipulation
import pandas as pd

# Read in data as pandas dataframe and display first 5 rows
dataset = pd.read_csv('/Users/samsonmihirette/Documents/SWELL-TEST/Final2/dataverse_all_events_OnTheFly.csv')

#rf=rf.replace(to_replace=" NULL",value=0)
dataset['MeanEntropy'] = dataset['MeanEntropy'].fillna(0)
dataset['MaxEntropy'] = dataset['MaxEntropy'].fillna(0)

y = dataset['Anxiety'].copy()
X = dataset.drop('Anxiety', axis=1).copy()
X = dataset.drop('Comfort', axis=1).copy()

def preprocess_inputs(df):
    df = df.copy()
    
    #df['condition'] = df['condition'].replace(label_mapping)
    
    y = df['Anxiety'].copy()
    X = df.drop('Anxiety', axis=1).copy()
    X = df.drop('Comfort', axis=1).copy()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = preprocess_inputs(dataset)


# Define classifiers and their respective parameter grids for Randomized Search
classifiers = {

    'LR': (LogisticRegression(), {
        'C': np.logspace(-4, 4, 20),  # Regularization parameter
        'penalty': ['l1', 'l2'],  # Penalty ('l1' for Lasso, 'l2' for Ridge)
        'solver': ['liblinear', 'saga']  # Optimization solver
    }),

    'LDA': (LinearDiscriminantAnalysis(), {
        'solver': ['lsqr', 'eigen'],  # Solvers compatible with shrinkage
        'shrinkage': [None, 'auto', 0.1, 0.5, 0.9]  # Shrinkage parameter (if used)
    }),
    
    'KNN': (KNeighborsClassifier(), {
        'n_neighbors': [5, 10, 15],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree']
    }),

    'CART': (DecisionTreeClassifier(), {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]          
    }),

    'RF': (RandomForestClassifier(), {
        'n_estimators': [100, 300, 500],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }),

    'GB': (GradientBoostingClassifier(), {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    }),

    'SVM': (SVC(), {
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto'],
        'kernel': ['linear', 'rbf']
    }),
    'XGB': (XGBRegressor(), {
        'n_estimators': [500, 600, 1000],
        'max_depth': [5, 6, 7],
        'learning_rate': [0.01, 0.1, 0.3],
        'subsample': [0.5, 0.7, 0.9],
        'colsample_bytree': [0.5, 0.7, 0.9],
        'gamma': [0, 1, 5]
    }),
    'MLP': (MLPRegressor(), {
            'hidden_layer_sizes': [(50, 50), (100, 100), (50,)],
            'activation': ['relu', 'tanh', 'logistic'],
    }),
    'abc': (AdaBoostClassifier(), {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1.0]
    }),
    'CB': (CatBoostClassifier(), {
        'iterations': [100, 300, 500],
        'learning_rate': [0.01, 0.1, 0.3],
        'depth': [4, 6, 8],
        'l2_leaf_reg': [1, 3, 5, 7, 9]
    }),
    'BN': (BayesianRidge(), {
        'alpha_1': [1e-6, 1e-7, 1e-8],
        'alpha_2': [1e-6, 1e-7, 1e-8],
        'lambda_1': [1e-6, 1e-7, 1e-8],
        'lambda_2': [1e-6, 1e-7, 1e-8]
    })

}

# Perform RandomizedSearchCV for each classifier
results = {}
for clf_name, (clf, param_grid) in classifiers.items():
    #print(f"Running Randomized Search for {clf_name}...")
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    clf_random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=10, cv=kf, scoring='f1_macro', random_state=42, n_jobs=-1)
    clf_random_search.fit(X, y)
    
    # Store cross-validation results, estimators, and test scores
    results[clf_name] = {
        'cv_results': clf_random_search.cv_results_,
        'best_estimator': clf_random_search.best_estimator_,
        'test_score': clf_random_search.best_score_
    }

# Save the model
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
 
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

save_obj(results, '/Users/samsonmihirette/Documents/SWELL-TEST/Final2/LFtrain_dataverse_RS-10cv')


/Users/samsonmihirette/opt/anaconda3/envs/myenv222/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/samsonmihirette/opt/anaconda3/envs/myenv222/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/samsonmihirette/opt/anaconda3/envs/myenv222/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/samsonmihirette/opt/anaconda3/envs/myenv222/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/samsonmihirette/opt/anaconda3/envs/myenv222/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

0:	learn: 1.5725111	total: 53.2ms	remaining: 5.27s
0:	learn: 1.5452679	total: 59.9ms	remaining: 5.93s
0:	learn: 1.5680033	total: 99.5ms	remaining: 9.85s
0:	learn: 1.6163795	total: 174ms	remaining: 17.3s
0:	learn: 1.6118880	total: 248ms	remaining: 24.6s
0:	learn: 1.5727907	total: 307ms	remaining: 30.4s
1:	learn: 1.3909904	total: 389ms	remaining: 19.1s
1:	learn: 1.4817133	total: 418ms	remaining: 20.5s
1:	learn: 1.4429891	total: 474ms	remaining: 23.2s
0:	learn: 1.5709534	total: 390ms	remaining: 38.6s
1:	learn: 1.3866427	total: 491ms	remaining: 24.1s
0:	learn: 1.6003192	total: 440ms	remaining: 43.6s
1:	learn: 1.4136198	total: 588ms	remaining: 28.8s
1:	learn: 1.4855082	total: 586ms	remaining: 28.7s
2:	learn: 1.0550924	total: 813ms	remaining: 26.3s
2:	learn: 1.1304460	total: 829ms	remaining: 26.8s
1:	learn: 1.3302426	total: 760ms	remaining: 37.2s
2:	learn: 1.0980095	total: 874ms	remaining: 28.3s
1:	learn: 1.4656636	total: 841ms	remaining: 41.2s
2:	learn: 1.0619712	total: 994ms	remaining: 32.

/Users/samsonmihirette/opt/anaconda3/envs/myenv222/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/samsonmihirette/opt/anaconda3/envs/myenv222/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/samsonmihirette/opt/anaconda3/envs/myenv222/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/samsonmihirette/opt/anaconda3/envs/myenv222/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._sco